In [6]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [7]:
# cd drive/MyDrive/IE7500_GroupB/Notebooks

In [8]:
# load necesary libraries
import numpy as np
import pandas as pd

In [9]:
dtypes_dict = {'headline': 'object',
               'url': 'object',
               'publisher': 'object',
               'stock': 'object',
               'tokens': 'object',
               'normalized_tokens': 'object',
               'filtered_tokens': 'object',
               'lemmas': 'object',
               'sentiment_score': 'float64',
               'Name': 'object',
               'Market Cap': 'float64',
               'Country': 'object',
               'IPO Year': 'float64',
               'Sector': 'object',
               'Industry': 'object',
               'year': 'int32',
               'month': 'int32',
               'day_of_week': 'int32',
               'sentiment_label': 'int64',
               'headline_length': 'int64',
               'word_count': 'int64',
               'Market_Cap_Category': 'object',
               'recommendation': 'object',
               'cap_Large': 'bool',
               'cap_Medium': 'bool',
               'cap_Mega': 'bool',
               'cap_Micro': 'bool',
               'cap_Nano': 'bool',
               'cap_Small': 'bool',
               'sector_Basic Materials': 'bool',
               'sector_Consumer Discretionary': 'bool',
               'sector_Consumer Staples': 'bool',
               'sector_Energy': 'bool',
               'sector_Finance': 'bool',
               'sector_Health Care': 'bool',
               'sector_Industrials': 'bool',
               'sector_Miscellaneous': 'bool',
               'sector_Real Estate': 'bool',
               'sector_Technology': 'bool',
               'sector_Telecommunications': 'bool',
               'sector_Utilities': 'bool', 'recommendation_label': 'int64',
               'publisher_label': 'int64', 'country_label': 'int64',
               'industry_label': 'int64'}

In [10]:
# load dataframes to use
df_main = pd.read_csv("saved_dfs/df_for_models.csv", dtype=dtypes_dict,
                      parse_dates=['date'])

In [11]:
df_main.head()

,headline,url,publisher,date,stock,tokens,normalized_tokens,filtered_tokens,lemmas,sentiment_score,...,sector_Industrials,sector_Miscellaneous,sector_Real Estate,sector_Technology,sector_Telecommunications,sector_Utilities,recommendation_label,publisher_label,country_label,industry_label
0,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01,A,"['Agilent', 'Technologies', 'Announces', 'Pric...","['agilent', 'technologies', 'announces', 'pric...","['agilent', 'technologies', 'announces', 'pric...","['agilent', 'technology', 'announces', 'pricin...",0.0000,...,True,False,False,False,False,False,1,4,45,18
1,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18,A,"['Agilent', '(', 'A', ')', 'Gears', 'Up', 'for...","['agilent', 'a', 'gears', 'up', 'for', 'q2', '...","['agilent', 'gears', 'q2', 'earnings', 'cards']","['agilent', 'gear', 'q2', 'earnings', 'card']",0.0000,...,True,False,False,False,False,False,1,16,45,18
2,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15,A,"['J.P.', 'Morgan', 'Asset', 'Management', 'Ann...","['morgan', 'asset', 'management', 'announces',...","['morgan', 'asset', 'management', 'announces',...","['morgan', 'asset', 'management', 'announces',...",0.3612,...,True,False,False,False,False,False,1,4,45,18
3,"Pershing Square Capital Management, L.P. Buys ...",http://www.gurufocus.com/news/1138704/pershing...,GuruFocus,2020-05-15,A,"['Pershing', 'Square', 'Capital', 'Management'...","['pershing', 'square', 'capital', 'management'...","['pershing', 'square', 'capital', 'management'...","['pershing', 'square', 'capital', 'management'...",0.0000,...,True,False,False,False,False,False,1,4,45,18
4,Agilent Awards Trilogy Sciences with a Golden ...,http://www.gurufocus.com/news/1134012/agilent-...,GuruFocus,2020-05-12,A,"['Agilent', 'Awards', 'Trilogy', 'Sciences', '...","['agilent', 'awards', 'trilogy', 'sciences', '...","['agilent', 'awards', 'trilogy', 'sciences', '...","['agilent', 'award', 'trilogy', 'science', 'go...",0.4588,...,True,False,False,False,False,False,1,4,45,18


Create Vectors from Headlines (Vectorization Models)

To generate vectors for the headline column, you can use different NLP vectorization techniques, including:

TF-IDF (Term Frequency - Inverse Document Frequency)

Count Vectorizer (Bag of Words)

Word2Vec (Pre-trained Word Embeddings)

Doc2Vec (Sentence-Level Embeddings)

BERT Embeddings (Transformer-based Representation)

# Goal 1: Predict Buy / Hold / Sell

## Method 4: MLP

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

2025-04-06 14:45:01.940840: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743965101.962533  930462 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743965101.969584  930462 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743965101.987611  930462 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743965101.987621  930462 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743965101.987624  930462 computation_placer.cc:177] computation placer alr

In [13]:
# Features and target
features = ["sentiment_score", "publisher_label", "country_label", "industry_label",
            "sector_Industrials", "sector_Miscellaneous", "sector_Real Estate",
            "sector_Technology", "sector_Telecommunications", "sector_Utilities"]
target = "recommendation_label"

# Split
X = df_main[features].fillna(0)
y = df_main[target]

# Train/test split
X_train, X_test, y_train_raw, y_test_raw = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert labels to one-hot for softmax classification
y_train = to_categorical(y_train_raw)
y_test = to_categorical(y_test_raw)
num_classes = y_train.shape[1]

In [14]:
model = Sequential([
    Dense(128, activation="relu", input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(num_classes, activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

/courses/IE7500.202530/shared/conda_env_1/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1743965110.443077  930462 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11437 MB memory:  -> device: 0, name: Tesla P100-PCIE-12GB, pci bus id: 0000:03:00.0, compute capability: 6.0


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,859 (38.51 KB)

 Trainable params: 9,859 (38.51 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=3,
    batch_size=256,
    verbose=1
)

Epoch 1/3


I0000 00:00:1743965113.557545  930593 service.cc:152] XLA service 0x14dcdc004d30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1743965113.557598  930593 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-12GB, Compute Capability 6.0
2025-04-06 14:45:13.626447: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1743965113.970075  930593 cuda_dnn.cc:529] Loaded cuDNN version 90300


  41/5768 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.6311 - loss: 0.8982   

I0000 00:00:1743965116.780918  930593 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5768/5768 ━━━━━━━━━━━━━━━━━━━━ 31s 5ms/step - accuracy: 0.9594 - loss: 0.1062 - val_accuracy: 0.9982 - val_loss: 0.0063
Epoch 2/3
5768/5768 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.9960 - loss: 0.0105 - val_accuracy: 0.9989 - val_loss: 0.0030
Epoch 3/3
5768/5768 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.9972 - loss: 0.0072 - val_accuracy: 0.9986 - val_loss: 0.0036


In [16]:
# Predict classes
y_pred_probs = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Classification report
print("MLP Keras Model Performance:")
print(classification_report(y_true_classes, y_pred_classes))

11535/11535 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
MLP Keras Model Performance:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     37064
           1       1.00      1.00      1.00    317370
           2       0.99      0.99      0.99     14678

    accuracy                           1.00    369112
   macro avg       1.00      1.00      1.00    369112
weighted avg       1.00      1.00      1.00    369112



In [17]:
model.save("keras_mlp_recommendation_model.h5")
print("Model saved as keras_mlp_recommendation_model.h5")

Model saved as keras_mlp_recommendation_model.h5


## Method 5: LSTM for Text-Based Prediction

In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import classification_report

In [19]:
# Convert lemmas list to space-separated strings
df_main["lemmas_str"] = df_main["lemmas"].apply(lambda x: " ".join(x) if isinstance(x, list) else "")

# Define text and target
texts = df_main["lemmas_str"].fillna("")
labels = df_main["recommendation_label"]

# Train/test split
X_train_text, X_test_text, y_train_raw, y_test_raw = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

# One-hot encode target
y_train = to_categorical(y_train_raw)
y_test = to_categorical(y_test_raw)
num_classes = y_train.shape[1]

In [20]:
# Tokenize text
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text)

# Convert to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_seq = tokenizer.texts_to_sequences(X_test_text)

# Pad sequences to the same length
max_len = 30
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding="post", truncating="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding="post", truncating="post")

In [21]:
embedding_dim = 100

model = Sequential([
    Embedding(input_dim=10000, output_dim=embedding_dim, input_length=max_len),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(32, activation="relu"),
    Dense(num_classes, activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

/courses/IE7500.202530/shared/conda_env_1/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Running 3 epochs to save time

In [22]:
history = model.fit(
    X_train_pad, y_train,
    validation_data=(X_test_pad, y_test),
    epochs=3,
    batch_size=512
)

Epoch 1/3
2884/2884 ━━━━━━━━━━━━━━━━━━━━ 47s 14ms/step - accuracy: 0.8580 - loss: 0.4987 - val_accuracy: 0.8598 - val_loss: 0.4906
Epoch 2/3
2884/2884 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - accuracy: 0.8609 - loss: 0.4871 - val_accuracy: 0.8598 - val_loss: 0.4905
Epoch 3/3
2884/2884 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - accuracy: 0.8611 - loss: 0.4858 - val_accuracy: 0.8598 - val_loss: 0.4927


In [23]:
# Predict classes
y_pred_probs = model.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Classification report
print("LSTM Model Performance on Lemmas:")
print(classification_report(y_true_classes, y_pred_classes))

11535/11535 ━━━━━━━━━━━━━━━━━━━━ 40s 3ms/step
LSTM Model Performance on Lemmas:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     37064
           1       0.86      1.00      0.92    317370
           2       0.00      0.00      0.00     14678

    accuracy                           0.86    369112
   macro avg       0.29      0.33      0.31    369112
weighted avg       0.74      0.86      0.80    369112



/courses/IE7500.202530/shared/conda_env_1/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/courses/IE7500.202530/shared/conda_env_1/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/courses/IE7500.202530/shared/conda_env_1/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [24]:
model.save("lstm_text_recommendation_model.h5")
print("Model saved as lstm_text_recommendation_model.h5")

Model saved as lstm_text_recommendation_model.h5


## Method 6: BERT + MLP (Transformer-Based)

In [25]:
!pip install transformers datasets

Defaulting to user installation because normal site-packages is not writeable


In [26]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import TruncatedSVD
from joblib import Parallel, delayed

/courses/IE7500.202530/shared/conda_env_1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Convert lemmas to joined string
df_main["lemmas_str"] = df_main["lemmas"].apply(lambda x: " ".join(x) if isinstance(x, list) else "")



In [29]:
# Convert lemmas to joined string
def get_bert_embedding(texts, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', truncation=True, padding=True, max_length=64)
        with torch.no_grad():
            outputs = bert_model(**inputs)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)


In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)

def get_bert_embedding(texts, batch_size=64):
    embeddings = []

    for i in tqdm(range(0, len(texts), batch_size), desc="Encoding BERT"):
        batch_texts = texts[i:i + batch_size]

        # Tokenize batch
        inputs = tokenizer(batch_texts, return_tensors='pt', truncation=True,
                           padding=True, max_length=64).to(device)

        # Disable gradient tracking and use inference mode for speed
        with torch.inference_mode():
            outputs = bert_model(**inputs)
            # Mean pooling over token dimension
            batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()

        embeddings.append(batch_embeddings)

    return np.vstack(embeddings)

In [36]:
from tqdm import tqdm

In [37]:
bert_vectors = get_bert_embedding(df_main["lemmas_str"].tolist())
np.save("bert_vectors.npy", bert_vectors)

Encoding BERT: 100%|██████████| 28837/28837 [19:23<00:00, 24.78it/s]  


In [38]:
# Convert BERT vectors to a feature matrix
X = np.load("bert_vectors.npy", allow_pickle=True)
y = df_main["recommendation_label"]
svd = TruncatedSVD(n_components=50, random_state=42)
X_reduced = svd.fit_transform(X)

In [39]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
# MLP classifier on top of BERT embeddings
clf = SGDClassifier(loss="log_loss", max_iter=100, random_state=42)
clf.fit(X_train, y_train)
# Predict
y_pred = clf.predict(X_test)
# Evaluate
print("BERT Embeddings + MLP Performance:")
print(classification_report(y_test, y_pred, zero_division=0))

BERT Embeddings + MLP Performance:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     37064
           1       0.86      1.00      0.92    317370
           2       0.00      0.00      0.00     14678

    accuracy                           0.86    369112
   macro avg       0.29      0.33      0.31    369112
weighted avg       0.74      0.86      0.80    369112

